In [1]:
#Open relevant packages
import requests
import argparse
import sys
import json
import pandas as pd
import numpy as np
from pprint import pprint
import re

In [2]:
#Define arguments
#Reference - https://github.com/Milchdealer/isoform-lister/blob/main/main.py 
sys.argv = [''] #to avoid unrecognised arguments error
parse = argparse.ArgumentParser()
#Adding arguments
parse.add_argument(
        "--searchlist",
        default="C:/unannotated_list.csv",
)
parse.add_argument(
        "--output",
        default="C:/unannotated_list_unip.csv",
)

_StoreAction(option_strings=['--output'], dest='output', nargs=None, const=None, default='C:/Users/Ananya/Documents/Edinburgh_Notes/Project/unannotated_list_unip.csv', type=None, choices=None, help=None, metavar=None)

In [3]:
un = parse.parse_args() #Parse arguments
terms = []
# Creating a list of UniProt IDs
with open(un.searchlist) as l:
    for line in l:
        terms.append(line)
fi = ['gene_orf', 'gene_primary', 'protein_name', 'go_c', 'xref_alphafolddb', 'xref_pdb']
a = ",".join(fi) #https://stackoverflow.com/questions/5299796/how-to-use-list-or-tuple-as-string-formatting-value
url = "https://rest.uniprot.org/uniprotkb/search?query=accession:{0}&fields={1}"
uni = []
for term in terms:
    results = requests.get(url.format(term, a)).text #Converting response into text
    data = json.loads(results) #To convert json string to dictionary; Reference - https://towardsdatascience.com/json-and-apis-with-python-fba329ef6ef0
    uni.extend(data['results']) #To add looped results into list

In [4]:
#pprint(uni)

In [5]:
#List comprehension for extracting UniProt IDs
ids = [item["primaryAccession"] for item in uni]
#print(ids)

In [6]:
# A function to extract nested dictionaries from the main list
def findkeys(node, kv): #https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-dictionaries-and-lists
    if isinstance(node, list):
        for i in node:
            for x in findkeys(i, kv):
                yield x
    elif isinstance(node, dict):
        if kv in node:
            yield node[kv]
        for j in node.values():
            for x in findkeys(j, kv):
                yield x

In [7]:
# A function to extract values based on keys
def find(key, dictionary): #https://gist.github.com/douglasmiranda/5127251
    for k, v in dictionary.items():
        if k == key:
            yield v
        elif isinstance(v, dict):
            for result in find(key, v):
                yield result
        elif isinstance(v, list):
            for d in v:
                for result in find(key, d):
                    yield result

In [8]:
woohoo = []
for j in range(len(uni)):
    yippee = list(find('value', uni[j]))
    woohoo.append(yippee[0])
#woohoo

In [13]:
sec = [] #https://www.youtube.com/watch?v=4wMKvr_rd8o
for dicti in uni:
    for key in dicti:
        if key == 'genes':
            manjhi = dicti[key][0] #Extract nested dictionary based on key
            ghoomar = ",".join(list(findkeys(manjhi, 'value'))) #Join values with comma separator
            sec.append(ghoomar)  
#sec

In [15]:
seca = []
for dicti in uni:
    for key in dicti:
        if key == 'proteinDescription':
            manjhi = dicti[key]
            pattern = re.compile(r"(\d*(?:\.[\d-]+))") #Regex pattern for identifying Enzyme Number
            pardesi = list(findkeys(manjhi, 'value'))
            edited = ",".join(s for s in pardesi if pattern.match(s))
            seca.append(edited)  
#seca

In [16]:
locs = list(findkeys(uni, 'uniProtKBCrossReferences'))
#print(locs)
c_locss = []
for i in range(len(uni)):
    #Join strings from dictionary which starts with specific pattern
    luca = ",".join(j for l in locs[i] for j in l['properties'][0]['value'].splitlines() if j.startswith('C:')) 
    c_locss.append(luca)

#c_locss

In [17]:
hallo = []
for dicti in uni:
    for key in dicti:
        if key == 'uniProtKBCrossReferences':
            manjhi = dicti[key]
            pattern = re.compile(r"^[1-9]+[A-Za-z]")
            pardesi = list(findkeys(manjhi, 'id'))
            edited = ",".join(s for s in pardesi if pattern.match(s))
            hallo.append(edited)  
#hallo

In [18]:
ditch = []
for dicti in uni:
    for key in dicti:
        if key == 'uniProtKBCrossReferences':
            manjhi = dicti[key]
            pattern = re.compile(r"^[Q,B]")
            pardesi = list(findkeys(manjhi, 'id'))
            edited = ",".join(s for s in pardesi if pattern.match(s))
            ditch.append(edited)  
#ditch

In [19]:
# Creating a dataframe and inserting columns with the data obtained so far
df1 = pd.DataFrame()
df1['UniProt Accession'] = ids
df1['Protein Name'] = woohoo
df1['Gene Name/Symbol'] = pd.Series(sec)
df1['Enzyme Number'] = pd.Series(seca)
df1['Subcellular locations'] = c_locss
df1['PDB Structure ID'] = pd.Series(hallo)
df1['AlphaFold Structure ID'] = pd.Series(ditch)
df1

,UniProt Accession,Protein Name,Gene Name/Symbol,Enzyme Number,Subcellular locations,PDB Structure ID,AlphaFold Structure ID
0,Q57YY8,"Dynein heavy chain, putative",Tb927.8.3250,,"C:axoneme,C:cilium,C:dynein complex,C:microtubule",,
1,Q38B11,"Protein kinase, putative",Tb10.70.0970,2.7.11.1,,,Q38B11
2,Q57UX4,Clu domain-containing protein,Tb927.8.4780,,"C:cilium,C:cytoplasm,C:flagellum attachment zone",,
3,Q386I5,DNA-directed DNA polymerase,Tb11.02.0770,2.7.7.7,C:mitochondrion,,Q386I5
4,Q587C0,3-hydroxy-3-methylglutaryl coenzyme A reductase,Tb927.6.4540,1.1.1.34,"C:ciliary basal body,C:endoplasmic reticulum m...",,
...,...,...,...,...,...,...,...
309,Q581D6,ATP pyrophosphate-lyase,Tb927.8.2550,,C:integral component of membrane,,
310,B3A0M1,Phosphatidylcholine:ceramide cholinephosphotra...,Tb11.02.0630,2.7.8.27,C:integral component of membrane,,B3A0M1
311,Q57XD4,PriCT_2 domain-containing protein,Tb10.61.2420,,"C:kinetoplast,C:mitochondrial matrix,C:nucleol...",,
312,Q386J8,"ABC transporter, putative",NaN,,"C:glycosome,C:glycosome membrane,C:integral co...",,Q386J8


In [20]:
# To add None when there is no information available
df1.where(df1.notnull(), None)

,UniProt Accession,Protein Name,Gene Name/Symbol,Enzyme Number,Subcellular locations,PDB Structure ID,AlphaFold Structure ID
0,Q57YY8,"Dynein heavy chain, putative",Tb927.8.3250,,"C:axoneme,C:cilium,C:dynein complex,C:microtubule",,
1,Q38B11,"Protein kinase, putative",Tb10.70.0970,2.7.11.1,,,Q38B11
2,Q57UX4,Clu domain-containing protein,Tb927.8.4780,,"C:cilium,C:cytoplasm,C:flagellum attachment zone",,
3,Q386I5,DNA-directed DNA polymerase,Tb11.02.0770,2.7.7.7,C:mitochondrion,,Q386I5
4,Q587C0,3-hydroxy-3-methylglutaryl coenzyme A reductase,Tb927.6.4540,1.1.1.34,"C:ciliary basal body,C:endoplasmic reticulum m...",,
...,...,...,...,...,...,...,...
309,Q581D6,ATP pyrophosphate-lyase,Tb927.8.2550,,C:integral component of membrane,,
310,B3A0M1,Phosphatidylcholine:ceramide cholinephosphotra...,Tb11.02.0630,2.7.8.27,C:integral component of membrane,,B3A0M1
311,Q57XD4,PriCT_2 domain-containing protein,Tb10.61.2420,,"C:kinetoplast,C:mitochondrial matrix,C:nucleol...",,
312,Q386J8,"ABC transporter, putative",None,,"C:glycosome,C:glycosome membrane,C:integral co...",,Q386J8


In [54]:
# To write the dataframe as a csv file
df1.to_csv(un.output) #https://stackoverflow.com/questions/69501185/save-a-list-of-tuple-to-csv-file-in-python